In [6]:
import math
import os
from glob import glob
from itertools import combinations
from urllib.error import URLError

import earthpy as et
import earthpy.appeears as etapp
import geopandas as gpd
import holoviews as hv
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import requests
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from IPython.display import Image
import utm
import xarray as xr
import xrspatial
import warnings
import zipfile


data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME, 'treebeard')
wbd_url = (
    'https://prd-tnm.s3.amazonaws.com'
    '/StagedProducts/Hydrography/WBD/HU2/Shape'
    '/WBD_10_HU2_Shape.zip')
wdb_shp_file = os.path.join(data_dir, 'WBD_10_HU2_Shape', 'Shape', 'WBDHU10.shp')

drapp_url = (
    'https://gisdata.drcog.org:8443'
    '/geoserver/DRCOGPUB/ows?'
    'service=WFS&version=1.0.0&'
    'request=GetFeature&'
    'typeName=DRCOGPUB:drapp_tile_scheme_2020'
    '&outputFormat=SHAPE-ZIP'
)
drapp_shp_file = os.path.join(data_dir, 'drapp_tile_scheme_2020', 'drapp_tile_scheme_2020.shp')

# Create the directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# Download the files
download_and_extract = False
if download_and_extract:
    wdb_resp = requests.get(wbd_url)
    drapp_resp = requests.get(drapp_url)

In [3]:
def extract_zip(target_dir, zip_filename, resp):
    zip_path = os.path.join(data_dir, zip_filename)
    with open(zip_path, 'wb') as f:
        f.write(resp.content)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(target_dir)

In [7]:
wbd_dir = os.path.join(data_dir, 'WBD_10_HU2_Shape')
drapp_dir = os.path.join(data_dir, 'drapp_tile_scheme_2020')
if download_and_extract:
    extract_zip(wbd_dir, 'WBD_10_HU2_Shape.zip', wdb_resp)
    extract_zip(drapp_dir, 'drapp_tile_scheme_2020.zip', drapp_resp)

In [8]:
wdb_shp_file

'/Users/ed.chan/earth-analytics/data/treebeard/WBD_10_HU2_Shape/Shape/WBDHU10.shp'

In [9]:
drapp_shp_file

'/Users/ed.chan/earth-analytics/data/treebeard/drapp_tile_scheme_2020/drapp_tile_scheme_2020.shp'

In [10]:
wdb_gdf = gpd.read_file(wdb_shp_file)
left_hand_creek_gdf = wdb_gdf[wdb_gdf['huc10'] == '1019000503'].copy()
study_area_gdf = wdb_gdf[wdb_gdf['huc10'].astype(str).str.startswith('10190005')].copy()

In [37]:
plot = left_hand_creek_gdf.hvplot(geo=True, color='blue', tiles='OSM',
                                alpha=0.3)
plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [38]:
left_hand_creek_gdf

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,huc10,name,hutype,humod,shape_Leng,shape_Area,ObjectID,geometry
48,{E27AC9D0-4FAC-486F-A1AA-667A4B584596},None,None,None,None,2012-10-24,None,46585.71,188.53,CO,1019000503,Left Hand Creek,S,None,None,None,49,"POLYGON ((-105.31911 40.17821, -105.31757 40.1..."


In [39]:
drapp_gdf = gpd.read_file(drapp_shp_file)
drapp_gdf = drapp_gdf.to_crs(left_hand_creek_gdf.crs) # EPSG:6428 -> EPSG:4269
drapp_gdf

,project,area,resolution,tile,photo_date,storage,geometry
0,2020,1a,"3 Inch, 6 Inch",S3E194c,20200426,drapparchive,"POLYGON ((-104.92568 39.30538, -104.92562 39.3..."
1,2020,1a,"3 Inch, 6 Inch",S3E194d,20200426,drapparchive,"POLYGON ((-104.91635 39.30533, -104.91629 39.3..."
2,2020,1a,"3 Inch, 6 Inch",S3E195c,20200426,drapparchive,"POLYGON ((-104.90702 39.30529, -104.90696 39.3..."
3,2020,1a,"3 Inch, 6 Inch",S3E195d,20200426,drapparchive,"POLYGON ((-104.89769 39.30524, -104.89763 39.3..."
4,2020,1a,"3 Inch, 6 Inch",S3E196c,20200426,drapparchive,"POLYGON ((-104.88836 39.30519, -104.88830 39.3..."
...,...,...,...,...,...,...,...
10130,2020,3,12 Inch,N4W379,20200711,drapparchive,"POLYGON ((-105.39078 40.20540, -105.39075 40.2..."
10131,2020,3,12 Inch,N4W348,20200711,drapparchive,"POLYGON ((-105.40962 40.24889, -105.40960 40.2..."
10132,2020,3,12 Inch,N4W349,20200711,drapparchive,"POLYGON ((-105.39071 40.24888, -105.39069 40.2..."
10133,2020,3,12 Inch,N4W338,20200711,drapparchive,"POLYGON ((-105.40960 40.26338, -105.40958 40.2..."


In [20]:
drapp_plot = drapp_gdf.hvplot(geo=True, color='red', tiles='OSM', alpha=0.3)
drapp_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [40]:
filtered_drapp_gdf = gpd.sjoin(drapp_gdf, left_hand_creek_gdf, how='inner', op='intersects')

/Users/ed.chan/miniconda3/envs/earth-analytics-python/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3488: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [41]:
filtered_drapp_plot = filtered_drapp_gdf.hvplot(geo=True, color='red', tiles='OSM', alpha=0.3)
filtered_drapp_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [42]:
len(filtered_drapp_gdf['tile'])

116

In [43]:
len(filtered_drapp_gdf['tile'].unique())

116

In [44]:
tile_base_url = 'https://drapparchive.s3.amazonaws.com/2020/'
tile_urls = [f'{tile_base_url}{tile}.tif' for tile in filtered_drapp_gdf['tile'].unique()]
# twf_urls = [f'{tile_base_url}{tile}.tfw' for tile in filtered_drapp_gdf['tile'].unique()]

In [46]:
drapp_tiles_dir = os.path.join(data_dir, 'drapp_tiles')
os.makedirs(drapp_tiles_dir, exist_ok=True)

In [47]:
def download_files(urls):
    count = 0 
    for url in urls:
        print(f"Downloading {url}...")
        filename = os.path.basename(url)
        path = os.path.join(drapp_tiles_dir, filename)
        if not os.path.exists(path):
            resp = requests.get(url)
            with open(path, 'wb') as f:
                f.write(resp.content)
        count += 1
        if count == 1:
            break

# download_files(twf_urls)
download_files(tile_urls)